In [58]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(r"C:\Users\asus\Downloads\volleyball_match.mp4")

prev_frame = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    hsv_frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)#trackbar here
    lower_yellow = np.array([10, 0, 0])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = cv.inRange(hsv_frame, lower_yellow, upper_yellow)

    yellow_regions = cv.bitwise_and(frame, frame, mask=yellow_mask)

    gray_yellow = cv.cvtColor(yellow_regions, cv.COLOR_BGR2GRAY)

    if prev_frame is not None:

        diff_frame = cv.absdiff(gray_yellow, prev_frame)

        _, thresholded = cv.threshold(diff_frame, 30, 255, cv.THRESH_BINARY)


        contours, _ = cv.findContours(thresholded, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv.contourArea(contour)
            if area < 100 or area > 500:
                continue

            x, y, w, h = cv.boundingRect(contour)
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv.imshow('Volleyball Tracking', frame)
        cv.imshow('Yellow Regions', yellow_regions)
        cv.imshow('Absolute Difference Mask with Morphology', thresholded)

    prev_frame = gray_yellow.copy()

    if cv.waitKey(20) & 0xFF == 27:  # esc
        break

cap.release()
cv.destroyAllWindows()
